In [19]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [20]:
elo_hard=pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date > '2021-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)

elo_clay=pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Clay where Date > '2021-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)


In [21]:
elo_data_hard =pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Hard where Date like '{time_now_formatted}' and resulted like 'False' --and Elo_Fav_Odds>1.19 --and Winner_Rank BETWEEN 1 AND 200 --and Winner_Rank<1000",
    con=devengine,)


elo_data_clay =pd.read_sql_query(
    f"Select DISTINCT * From Elo_AllMatches_Clay where Date like '{time_now_formatted}' and resulted like 'False' --and Elo_Fav_Odds>1.19",
    con=devengine,)

In [22]:
def get_player_record(player,opponent_rank,history,time,opponent,rank):
    player_history_top50=pd.DataFrame()
    player_history_top100=pd.DataFrame()
    player_history=pd.DataFrame()
    if opponent_rank<50:
        player_history_top50=history[(((history['Fav']==player)&(history['Dog_Rank']<50))|((history['Dog']==player)&(history['Fav_Rank']<50)))]
        #if player_history_top50.empty==False and (len(player_history_top50[player_history_top50['Winner']==player])/len(player_history_top50))  > 0.5:
            
            #print(time,player,"Top50",len(player_history_top50))
            #print(len(player_history_top50[player_history_top50['Winner']==player])/len(player_history_top50))        
    if opponent_rank<100:
        player_history_top100=history[(((history['Fav']==player)&(history['Dog_Rank']<100))|((history['Dog']==player)&(history['Fav_Rank']<100)))]
        #if player_history_top100.empty==False and (len(player_history_top100[player_history_top100['Winner']==player])/len(player_history_top100))   > 0.5:     
        #    print(time,player,"Top100",len(player_history_top100))
        #    print(len(player_history_top100[player_history_top100['Winner']==player])/len(player_history_top100))   
            
    else:
        opponent_rank_low=opponent_rank-50
        opponent_rank_high=opponent_rank+50
        player_history=history[((history['Fav']==player)&((history['Dog_Rank']>opponent_rank_low)&(history['Dog_Rank']<opponent_rank_high)))|
        ((history['Dog']==player)&((history['Fav_Rank']>opponent_rank_low)&(history['Fav_Rank']<opponent_rank_high)))]
        #if player_history.empty==False:
        #    if (len(player_history[player_history['Winner']==player])/len(player_history)) > 0.5 and len(player_history) >5:
        #        print(time,player,opponent_rank_low,opponent_rank_high,len(player_history))
        #        print(len(player_history[player_history['Winner']==player])/len(player_history))       
    
    return player_history,player_history_top100,player_history




In [23]:
def get_player_record(player, opponent_rank, history, range_low,range_high,auto):
    if auto:
        opponent_rank_low = opponent_rank - range_low
        opponent_rank_high = opponent_rank + range_high
    else:
        opponent_rank_low = range_low
        opponent_rank_high = range_high

    player_history = history[
        (
            (history["Fav"] == player)
            & (
                (history["Dog_Rank"] > opponent_rank_low)
                & (history["Dog_Rank"] < opponent_rank_high)
            )
        )
        | (
            (history["Dog"] == player)
            & (
                (history["Fav_Rank"] > opponent_rank_low)
                & (history["Fav_Rank"] < opponent_rank_high)
            )
        )
    ]
    if player_history.empty == False:
        result = float(
            len(player_history[player_history["Winner"] == player])
            / len(player_history)
        )
        return result, len(player_history)
    else:
        return 0, 0


In [24]:
result_df=pd.DataFrame()
for _,row in elo_data_hard.sort_values(by='Time').iterrows():
    low_limit=50
    high_limit=50

    percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_hard,low_limit,high_limit,True)
    count=0
    while games <10 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_hard,low_limit,high_limit,True)

    low_limit=50
    high_limit=50
    percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_hard,low_limit,high_limit,True)
    count=0
    while games2 <5 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_hard,low_limit,high_limit,True)       
        
    if True and games > 4 and games2 > 4:# and ((percent >= 0.6) | (percent2 > 0.5)):
        # if  (percent2 < 5)| (percent < 5):
        temp_df = pd.DataFrame(
            {   "Time":[row.Time],
                "Fav": [row.Fav],
                "Fav_Record": ["{:.0%}".format(percent)],
                "Fav_Games": [games],
                "Dog": [row.Dog],
                "Dog_Record": ["{:.0%}".format(percent2)],
                "Dog_Games": [games2],
            }
        )

        result_df=pd.concat([result_df,temp_df])
        #print(row.Time, row.Fav, row.Fav_Odds, row.Dog, row.Dog_Odds)
        #print(
        #    "{:.0%}".format(percent),
        #    f"({games} Games)",
        #    "{:.0%}".format(percent2),
        #    f"({games2} Games)",
        #)
#result_df      


In [ ]:
serving=pd.read_excel('servers_today.xlsx')
serving_womens=pd.read_excel('servers_today_womens.xlsx')
serving=pd.concat([serving,serving_womens])
serving=serving.drop(columns='Time')
pd.merge(result_df,serving,how='left',left_on=['Fav','Dog'],right_on=['Fav','Dog'])#.to_clipboard(index=False)

,Time,Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,Fav_Serve%,Dog_Return%,Dog_Serve%,Fav_Return%
0,11:10,Christopher Oconnell,58%,12,Yosuke Watanuki,43%,7,NaN,21.0,80.0,NaN
1,11:40,Marc Polmans,80%,10,Hsiou Hsu Yu,67%,6,71.0,NaN,NaN,23.0
2,11:45,Makoto Ochi,45%,11,Savriyan Danilov,40%,5,NaN,NaN,NaN,NaN
3,14:00,Yuta Shimizu,50%,10,Andrew Harris,0%,5,38.0,15.0,66.0,13.0
4,18:00,Valentin Royer,80%,10,Alexis Gautier,40%,5,NaN,NaN,NaN,NaN
5,18:30,Benjamin Hassan,55%,11,Karl Friberg,60%,5,NaN,NaN,NaN,NaN
6,19:30,Alexandar Lazarov,64%,11,Elliot Benchetrit,50%,6,67.0,12.0,62.0,12.0
7,20:00,Illya Marchenko,45%,11,Duje Ajdukovic,80%,5,75.0,38.0,60.0,21.0
8,21:00,Antoine Hoang,10%,10,Marko Topo,20%,5,70.0,7.0,57.0,21.0
9,21:30,Cem Ilkel,70%,10,Denis Istomin,50%,6,68.0,18.0,79.0,11.0


In [ ]:
result_df=pd.DataFrame()
for _,row in elo_data_clay.sort_values(by='Time').iterrows():
    low_limit=50
    high_limit=50

    percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_clay,low_limit,high_limit,True)
    count=0
    while games <10 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent,games=get_player_record(row.Fav,row.Dog_Rank,elo_clay,low_limit,high_limit,True)

    low_limit=50
    high_limit=50
    percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_clay,low_limit,high_limit,True)
    count=0
    while games2 <5 and count<200:
        count=count+1
        low_limit=low_limit+10
        high_limit=high_limit+10
        percent2,games2=get_player_record(row.Dog,row.Fav_Rank,elo_clay,low_limit,high_limit,True)       
        
    if True and games > 4 and games2 > 4 and ((percent >= 0.5) & (percent2 < 0.4)):
        # if  (percent2 < 5)| (percent < 5):
        temp_df = pd.DataFrame(
            {   "Time":[row.Time],
                "Fav": [row.Fav],
                "Fav_Record": ["{:.0%}".format(percent)],
                "Fav_Games": [games],
                "Dog": [row.Dog],
                "Dog_Record": ["{:.0%}".format(percent2)],
                "Dog_Games": [games2],
            }
        )

        result_df=pd.concat([result_df,temp_df])
        #print(row.Time, row.Fav, row.Fav_Odds, row.Dog, row.Dog_Odds)
        #print(
        #    "{:.0%}".format(percent),
        #    f"({games} Games)",
        #    "{:.0%}".format(percent2),
        #    f"({games2} Games)",
        #)
result_df      


""


In [ ]:
serving=pd.read_excel('servers_today.xlsx')
serving_womens=pd.read_excel('servers_today_womens.xlsx')
serving=pd.concat([serving,serving_womens])
serving=serving.drop(columns='Time')
pd.merge(result_df,serving,how='left',left_on=['Fav','Dog'],right_on=['Fav','Dog'])#.to_clipboard(index=False)

KeyError: 'Fav'